<a href="https://colab.research.google.com/github/heathjohn62/CS155-Fake-Deep/blob/main/project3/mp3_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Each LINE in dataset is a line in resulting dataframe (numbers included)
# Removes punctuation
# To access line i, use train_data[0][i] 
# train_data is a pandas DataFrame (2309 rows x 1 column)
# Each sonnet has a number in the line before it begins

def get_data_no_punc():
    train_data = pd.read_csv('https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project3/data/shakespeare.txt', delimiter='\n', header=None)

    punctuation = [',']
    # Go through each line of dataframe
    for i in range(len(train_data[0])):
        
        train_data[0][i] = train_data[0][i].replace(',', '')
        train_data[0][i] = train_data[0][i].replace('.', '')
        train_data[0][i] = train_data[0][i].replace('!', '')
        train_data[0][i] = train_data[0][i].replace('?', '')
        train_data[0][i] = train_data[0][i].replace(';', '')
        train_data[0][i] = train_data[0][i].replace(':', '')
        train_data[0][i] = train_data[0][i].replace('(', '')
        train_data[0][i] = train_data[0][i].replace(')', '')
        train_data[0][i] = train_data[0][i].replace('\'', '') # remove apostrophe
        # Remove leading and trailing zeros
        train_data[0][i] = train_data[0][i].strip().lower()

        # Convert line to array of strings
        train_data[0][i] = train_data[0][i].split(' ')

    return train_data

In [3]:
# Get syllable dictionary as dataframe
def get_syl_dict():
    # Split on NEW LINES
    syllable_dict = pd.read_csv('https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project3/data/Syllable_dictionary.txt', delimiter='\n', header=None, names=['words'])


    for i in range(len(syllable_dict['words'])):
        temp = syllable_dict['words'][i].split(" ", 1)[0]
        # remove apostrophe from syllable dictionary words
        syllable_dict['words'][i] = temp.replace('\'', '')
    return syllable_dict

In [4]:
# Map each word to its ID (word_id_map) and each ID to its work (id_word_map)
# Length of map: 3176 (excluding duplicate words when apostrophes removed)
def get_word_id_map(dict):
    word_id_map = {}
    word_counter = 0
    id_word_map = {}

    for word in dict['words']:
        # Fix indexing for duplicate words
        if word in word_id_map:
            word_counter -= 1
        word_id_map[word] = word_counter
        id_word_map[word_counter] = word
        word_counter += 1

    return word_id_map, id_word_map

In [5]:
# Map each word to all words that come after it
def get_next_word_map(id_map, data):
    next_word_map = {}
    ind = 1

    # Convert training data to 1D list
    poems = data.values.flatten().tolist()
    poems = sum(poems, [])


    for word in poems[ind:]:
        key = poems[ind - 1]

        # Ensure neither word nor key are numbers (denoting new poem)
        if (not word.isdigit()) and (not key.isdigit()):
            key_id = id_map[key]
            # Add key, word pair to map
            if key_id in next_word_map:
                next_word_map[key_id].append(id_map[word])
            else:
                next_word_map[key_id] = [id_map[word]]
        ind += 1

    return next_word_map

In [6]:
# MAIN CODE - call methods to get desired dataset, dictionary, and maps
# Training data with punctation removed, separated by lines
train_data = get_data_no_punc()
# Dictionary mapping words to their number of syllables
syllable_dict = get_syl_dict()
# Dictionary mapping words to their unique IDs
word_id_map, id_word_map = get_word_id_map(syllable_dict)
# Dictionary mapping words to 
next_word_map = get_next_word_map(word_id_map, train_data)